In [1]:
from linkedin_api import Linkedin
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
user_name=os.getenv('LINKEDIN_NAME')
user_pass=os.getenv('LINKEDIN_PASSWORD')

api=Linkedin(user_name,user_pass)
print(dir(api))

['_MAX_POST_COUNT', '_MAX_REPEATED_REQUESTS', '_MAX_SEARCH_COUNT', '_MAX_UPDATE_COUNT', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_cookies', '_fetch', '_get_list_feed_posts_and_list_feed_urns', '_headers', '_post', 'add_connection', 'client', 'follow_company', 'get_company', 'get_company_updates', 'get_conversation', 'get_conversation_details', 'get_conversations', 'get_current_profile_views', 'get_feed_posts', 'get_invitations', 'get_job', 'get_job_skills', 'get_post_comments', 'get_post_reactions', 'get_profile', 'get_profile_connections', 'get_profile_contact_info', 'get_profile_experiences', 'get_profile_member_badges', 'get_

In [3]:
feed_result=api.get_profile_posts(public_id='areganti')

In [4]:
print(feed_result)

[{'actor': {'urn': 'urn:li:member:413250064', 'image': {'accessibilityTextAttributes': [], 'attributes': [{'sourceType': 'PROFILE_PICTURE', 'miniProfile': {'firstName': 'Aishwarya', 'lastName': 'Naresh Reganti', 'dashEntityUrn': 'urn:li:fsd_profile:ACoAABihshABDFVFq1kZ0vYGaHic4b9X5Sq80DE', 'occupation': 'Tech Lead @ AWS | Lecturer | Advisor | Researcher | Speaker | Investor | CMU LTI Alumni |', 'objectUrn': 'urn:li:member:413250064', 'entityUrn': 'urn:li:fs_miniProfile:ACoAABihshABDFVFq1kZ0vYGaHic4b9X5Sq80DE', 'backgroundImage': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 640, 'fileIdentifyingUrlPathSegment': '200_800/profile-displaybackgroundimage-shrink_200_800/0/1732038424972?e=1747872000&v=beta&t=EYx2aKz_MgiIlgr3fyT1XtZAcxMeYJFjSroTccEMwms', 'expiresAt': 1747872000000, 'height': 160}, {'width': 640, 'fileIdentifyingUrlPathSegment': '350_1400/profile-displaybackgroundimage-shrink_350_1400/0/1732038424972?e=1747872000&v=beta&t=a57o42njQykWVoKfu0UANNqPUx4Dlv6DE5XE7AJE

In [5]:
for result in feed_result:
    text=result['commentary']['text']['text']
    print(text)
    print("-"*80)



😅 There are painfully few people in the world who have both — the experience of building, scaling, and running a multi-billion dollar business, *and* a deep understanding of AI and how it can drive real business success.

Most seasoned leaders and C-suite executives are fatigued, struggling to understand AI and keep up with its pace. On the other hand, those who understand AI deeply are often young, still figuring out how to scale and build businesses that create meaningful value. 

🫡 That’s exactly why I respect Gajen Kandiah so much. He bridges both worlds effortlessly.

Over time, I’ve had the chance to talk to him about so much — from how agents work to reasoning model architectures, and even the differences across AI development platforms. And every single time, I’m amazed at how deeply he understands AI and how easily he cuts through the hype. 

⛳ He gets into so much detail that I often find myself thinking, wait, how does the big man know all this, with a thousand other things 